In [7]:
import pandas as pd
import numpy as np

In [9]:
df=pd.read_csv("Facultad_ingenieria.csv")
df.columns
df["ASIGNATURA "].fillna(method="ffill",inplace=True)
df.groupby("ASIGNATURA ").sum().to_clipboard()
df_melt=df.melt(id_vars=["ASIGNATURA "],var_name="Docente")
df_melt=df_melt[(df_melt["value"]>0) & (df_melt.Docente!="Unnamed: 24")]

In [10]:
set(df_melt['ASIGNATURA '])
df_melt["code"]=df_melt["ASIGNATURA "].apply(lambda x: x[:9])

# Creamos un diccionario con la relación código asignatura - nombre asignatura
dict_courses=df_melt.set_index("code").to_dict()["ASIGNATURA "]


In [17]:
oferta_x_curso={i:np.random.choice([1,2],p=[0.9,0.1]) for i in set(df_melt["code"])}

In [34]:
L={str(i):[str(i)+"-"+str(j)  for j in range(1,oferta_x_curso[i]+1)] for i in dict_courses.keys()}
grupos=[]
for l in L.values():
    for ele in l:
        grupos.append(ele)

curso_base={str(i)+"-"+str(j):str(i)  for i in dict_courses.keys() for j in range(1,oferta_x_curso[i]+1) }


In [28]:
horas_grupo={i:3 for i in grupos}
max_horas_prof={i:np.random.choice([8,16,20],p=[0.25,0.5,0.25]) for i in set(df_melt["Docente"])}

In [35]:
dict_profs=df_melt.groupby("Docente").agg(list)[["code"]].reset_index()
dict_profs=dict_profs.set_index("Docente").to_dict()["code"]


In [36]:
df_melt["id-tuple"]=df_melt.apply(lambda x: (x["Docente"],x["code"]),axis=1)
theta=df_melt.set_index("id-tuple").to_dict()["value"]


In [37]:
num_grupos={1:1,2:3}

In [47]:
max_cursos=6
I=set(dict_profs.keys())
J=set(grupos)   
K=set(dict_courses.keys())
C=range(0,max_cursos+1)

tuplas=[(i,j) for i in I for j in J if (i,curso_base[j]) in theta.keys()]
prof_curso=[e for e in theta.keys()  ]
# posibles combinaciones profeso y cantidad de cursos
IC=[(i,c) for i in I for c in C]

In [57]:

import pulp as plp

# Create the 'prob' variable to contain the problem data
prob = plp.LpProblem("Modelo William", plp.LpMaximize)

# Create problem variables from dictionary
# Docente i es asignado a grupo j (binaria)
x=plp.LpVariable.dicts("x",tuplas,0,1,plp.LpBinary)

# Docente i es asignado a c cursos diferentes
N=plp.LpVariable.dicts("N",IC,0,1,plp.LpBinary)

# Un profesor es asignado a un curso
z=plp.LpVariable.dicts("z",theta.keys(),0,1,plp.LpBinary)

# e=(i,j) representa la tupla
prob+=plp.lpSum([x[(i,j)]*theta[i,curso_base[j]] for (i,j) in tuplas])

# Cada curso tiene un docente asignado
for j in J:
    prob+=plp.lpSum([x[i,j] for i in I if (i,j) in tuplas])==1

# Contar cuantos cursos diferentes da un profesor
for i in I:
    prob+=plp.lpSum([z[(i,k)] for k in K if (i,k) in theta.keys()])==plp.lpSum([l*N[i,l] for l in C])

# el profesor tiene una cantidad de cursos asignada
for i in I:
    prob+=plp.lpSum([N[i,l] for l in C])==1

# el profesor no puede dar mas cursos de los que tiene asignados
for i in I:
    for k in K:
        if (i,k) in theta.keys():
            plp.lpSum([x[(i,j)] for j in L[k] if (i,j) in tuplas])<=z[(i,k)]*len(L[k])

    
prob.solve()


c:\Users\fanca\.conda\envs\an_env\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [53]:
L

{'8830-0005': ['8830-0005-1'],
 '8830-0004': ['8830-0004-1'],
 '8830-0008': ['8830-0008-1'],
 '8830-0006': ['8830-0006-1'],
 '8830-0007': ['8830-0007-1'],
 '9937-0002': ['9937-0002-1'],
 '8827-0008': ['8827-0008-1'],
 'Electiva ': ['Electiva -1'],
 '6069-0012': ['6069-0012-1'],
 '9937-0001': ['9937-0001-1'],
 '9937-0003': ['9937-0003-1'],
 '9937-0005': ['9937-0005-1'],
 '9937-0011': ['9937-0011-1'],
 '9937-0007': ['9937-0007-1'],
 '9937-0012': ['9937-0012-1', '9937-0012-2'],
 '8830-0001': ['8830-0001-1'],
 '8830-0010': ['8830-0010-1'],
 '8833-0021': ['8833-0021-1'],
 '8830-0012': ['8830-0012-1'],
 '5555- 001': ['5555- 001-1'],
 'Optativa ': ['Optativa -1'],
 '5566-0009': ['5566-0009-1'],
 '6612-0001': ['6612-0001-1'],
 '6069-0011': ['6069-0011-1'],
 '9937-0013': ['9937-0013-1'],
 '6069-0127': ['6069-0127-1'],
 '8821-0010': ['8821-0010-1'],
 '9935-0004': ['9935-0004-1'],
 '8828-0028': ['8828-0028-1'],
 '8830-0009': ['8830-0009-1'],
 '9937-0014': ['9937-0014-1'],
 '8830-0011': ['8830-001